# Programs

# Programs vs modules

* Programs are modules.
* "Program" modules can be imported like any other module.
* Code not within a function or class is executed.
* Best practices:
  * Write a `main(args)` function.
  * Call `main()` withoin `if __name__ == '__main__'` to prevent it from being called on `import`.

## Example programm: copytool

```
usage: copytool.py [-h] [--recursive] [--preserve {access,all,time,none}]
                   SOURCE [SOURCE ...] TARGET

copy files or folders

positional arguments:
  SOURCE                file or folder to copy
  TARGET                target file or folder

optional arguments:
  -h, --help            show this help message and exit
  --recursive, -r       recursively copy folders
  --preserve {access,all,time,none}, -p {access,all,time,none}
                        preserve specified attributes, default: all
```

## Basic structure

```python
"""
Command line utility to copy files.
"""
import argparse
import logging
import sys

_log = logging.getLogger('copytool')


def main(args=None):
    pass # TODO: Implement.


if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    sys.exit(main())
```

## Actual processing

Implement a function that does the actual job:

In [1]:
def copy(source, target, preserve='all', is_recursive=False):
    _log.info('copy %s to %s', source, target)
    # TODO: Actually copy files using shutil.

A complete implementation can be done quite easily using various functions in `shutil`.

## Command line arguments

* The `argparse` module has a class `ArgumentParser` to represent command line arguments.
* Possible arguments can be declared using `ArgumentParser.add_argument()`.
* User arguments can be parsed using `ArgumentParser.parse_args()`.
* `--help ` is automatically available and prints an online help and exits.
* `--version` can easily be added to print the version number and exit.
* Argument errors cause the program to print an error message and exit.

## Parsing arguments


Set up an `ArgumentParser` with a short description of the program:

In [2]:
import argparse
parser = argparse.ArgumentParser(description='copy files or folders')

## Parsing arguments (continued)

Add one or more `SOURCE` path (`nargs='+'`):

In [3]:
parser.add_argument(
    'sources', metavar='SOURCE', nargs='+',
    help="file or folder to copy")

_StoreAction(option_strings=[], dest='sources', nargs='+', const=None, default=None, type=None, choices=None, help='file or folder to copy', metavar='SOURCE')

## Parsing arguments (continued)

Add a single target path:

In [4]:
parser.add_argument(
    'target', metavar='TARGET',
    help="target file or folder")

_StoreAction(option_strings=[], dest='target', nargs=None, const=None, default=None, type=None, choices=None, help='target file or folder', metavar='TARGET')

## Parsing arguments (continued)

Add a boolean switch `--recursive` that can be enabled to copy files recursively:

In [5]:
parser.add_argument(
    '--recursive', '-r', action='store_true',
    help='recursively copy folders')

_StoreTrueAction(option_strings=['--recursive', '-r'], dest='recursive', nargs=0, const=True, default=False, type=None, choices=None, help='recursively copy folders', metavar=None)

## Parsing arguments (continued)

Add an optional choice `--preserve` that can specify which file attributes to preserve. If no `--preserve` ist specified, the default value is `'all'`. Note that the default value can also be referred to in the `help` argument using `'%(default)s'`.

In [6]:
parser.add_argument(
    '--preserve', '-p',
    choices=['access', 'all', 'time', 'none'],
    default='all',
    help='preserve specified attributes, default: %(default)s')

_StoreAction(option_strings=['--preserve', '-p'], dest='preserve', nargs=None, const=None, default='all', type=None, choices=['access', 'all', 'time', 'none'], help='preserve specified attributes, default: %(default)s', metavar=None)

## Parsing arguments (continued)

Add `--version` to show the version number and exit:

In [7]:
parser.add_argument(
     '--version', action='version', version='%(prog)s 1.0')

_VersionAction(option_strings=['--version'], dest='version', nargs=0, const=None, default='==SUPPRESS==', type=None, choices=None, help="show program's version number and exit", metavar=None)

## Parsing arguments (continued)

Now we can parse some arguments:

In [8]:
arguments = parser.parse_args([
    '--recursive', 'data/customers.csv', 'data/products.csv', '/tmp'
])
print(arguments.sources)
print(arguments.target)
print(arguments.preserve)
print(arguments.recursive)

['data/customers.csv', 'data/products.csv']
/tmp
all
True


# Error handling

* `main()` returns 0 on success and 1 on error.
* Error in command line arguments are handled by `ArgumentParser`, which automatically calls `sys.exit(2)`.
* Best practice:
  * For situations that can be fixed by the user, show a simple message.
  * For situations that must be fixed by the developer, show a message with a stack trace.

## Error handling: example

```python
    exit_code = 1
    try:
        for source in arguments.sources:
            copy(
                source, arguments.target,
                is_recursive=arguments.recursive,
                preserve=arguments.preserve)
        exit_code = 0  # Success!
    except KeyboardInterrupt:
        _log.error('stopped as requested by user')
    except (OSError) as error:
        _log.error(error)
    except Exception as error:
        _log.exception(error)
    return exit_code
```

#  Summary

* Programs are modules, too.
* Write a separate `main()` and functions for the actual processing.
* Use `argparse.ArgumentParser` to handle command line arguments.
* Use `sys.exit()` to return an exit code: 0 = sucess, 1 or more = error.
* For errors the use can fix, only show a message but no stack trace.